In [1]:
import tf2onnx
import os
import tensorflow as tf


# Custom loss function (from your original code)
def combined_loss(y_true, y_pred):
    """
    Calculates the mean squared error loss between true and predicted values.
    """
    mse_loss = tf.keras.losses.mean_squared_error(y_true, y_pred)
    return mse_loss

# Define the path to your Keras model and the output ONNX model
keras_model_path = "../models/model_e50_b4.h5"
onnx_model_path = "../models/model_e50_b4.onnx"

# Ensure the Keras model file exists
if not os.path.exists(keras_model_path):
    print(f"Error: Keras model file not found at {keras_model_path}")
else:
    try:
        # Load the Keras model
        keras_model = tf.keras.models.load_model(keras_model_path, custom_objects={'combined_loss': combined_loss})
        print(f"Keras model loaded successfully from: {keras_model_path}")
        
        # Convert the Keras model to ONNX
        model_proto, _ = tf2onnx.convert.from_keras(keras_model, output_path=onnx_model_path)
        print(f"\nSuccessfully converted Keras model to ONNX at: {onnx_model_path}")
    except Exception as e:
        print(f"Error during ONNX conversion: {e}")

import onnx
onnx_model_path = "../models/model_e50_b4.onnx"
onnx_model = onnx.load(onnx_model_path)
onnx.checker.check_model(onnx_model)
print("ONNX model is valid.")



Keras model loaded successfully from: ../models/model_e50_b4.h5

Successfully converted Keras model to ONNX at: ../models/model_e50_b4.onnx
ONNX model is valid.


In [ ]:
## guide to convert ONNX model to TensorRT engine
## note: tersorRT must be installed and configured properly at first. 
# converting model.onnx to TensorRT engine
# pip install tensorrt pycuda

# cd models
# trtexec --onnx=model_e50_b4.onnx --saveEngine=model_e50_b4.trt
# trtexec --onnx=model_e50_b4.onnx --saveEngine=model_e50_b4.trt --fp16 --verbose

In [9]:
import cv2
import numpy as np
import imutils
import os
import onnxruntime
import torch

# Adjusted threshold for overall frame anomaly detection
ANOMALY_THRESHOLD = 0.00435

# Threshold for identifying anomalous regions within a frame
ANOMALY_REGION_PIXEL_THRESHOLD = 0.200

onnx_model_path = "../models/model_e50_b4.onnx"
video_path = "D:\\Research Dataset\\Surveillance\\Test\\testing_video.mp4"

# Ensure output folders exist
if not os.path.exists("output_frames"):
    os.makedirs("output_frames")
if not os.path.exists("reconstructed_frames"):
    os.makedirs("reconstructed_frames")
if not os.path.exists("anomaly_maps"):
    os.makedirs("anomaly_maps")

frame_count_processed = 0
anomaly_count = 0

def mean_squared_loss(x1, x2):
    """
    Calculates the true Mean Squared Error (MSE) between two arrays.
    """
    x1_np = x1 if isinstance(x1, np.ndarray) else x1.cpu().numpy()
    x2_np = x2 if isinstance(x2, np.ndarray) else x2.cpu().numpy()
    difference = x1_np.flatten() - x2_np.flatten()
    sq_difference = difference ** 2
    mse = np.mean(sq_difference)
    return mse

def preprocess_frame_for_onnx_pytorch(frame):
    """
    Preprocess frame for ONNX and returns a PyTorch tensor (NHWC).
    """
    frame_resized = cv2.resize(frame, (232, 232), interpolation=cv2.INTER_AREA)
    gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY).astype(np.float32) / 255.0
    normalized_gray = (gray - 0.5) * 2.0
    prep_frame = np.expand_dims(normalized_gray, axis=-1)
    model_input_np = np.expand_dims(prep_frame, axis=0)
    # Convert NumPy array to PyTorch tensor
    model_input_torch = torch.from_numpy(model_input_np).float()
    return model_input_torch

# Load the ONNX model using onnxruntime
try:
    providers = ['CUDAExecutionProvider', 'CPUExecutionProvider']
    ort_session = onnxruntime.InferenceSession(onnx_model_path, providers=providers)
    input_name = ort_session.get_inputs()[0].name
    output_name = ort_session.get_outputs()[0].name
    print("ONNX model loaded successfully.")
    print(f"Expected input shape for '{input_name}': {ort_session.get_inputs()[0].shape}")

    # Check the actual provider used
    print(f"Using device: {ort_session.get_providers()}")

except Exception as e:
    print(f"Error loading ONNX model: {e}")
    exit()

# Open the video file for processing
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print(f"Error: Could not open video file at {video_path}")
    exit()

length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"Total frames in video: {length}")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("End of video or issue reading frame.")
        break

    frame_count_processed += 1

    original_frame_color = imutils.resize(frame, width=700, height=600)

    # Preprocess frame for ONNX and get a PyTorch tensor
    processed_frame_torch = preprocess_frame_for_onnx_pytorch(frame)
    processed_frame_onnx = processed_frame_torch.numpy() # Convert to NumPy for onnxruntime

    # Run inference with the ONNX model
    ort_inputs = {input_name: processed_frame_onnx}
    ort_outputs = ort_session.run([output_name], ort_inputs)
    reconstructed_frame_onnx = ort_outputs[0]
    reconstructed_frame_torch = torch.from_numpy(reconstructed_frame_onnx).float() # Convert output to PyTorch tensor

    # The output of the ONNX model is now a PyTorch tensor (reconstructed_frame_torch)
    # and the input (as a tensor) was processed_frame_torch.

    # Calculate the overall reconstruction loss (MSE) using NumPy for consistency
    loss = mean_squared_loss(processed_frame_onnx, reconstructed_frame_onnx)
    loss_display = f"{loss:.5f}"
    print(f"Frame {frame_count_processed} Loss (ONNX): {loss_display}")

    display_frame = original_frame_color.copy()
    status_text = f"Normal: {loss_display}"
    color = (0, 255, 0)

    if loss > ANOMALY_THRESHOLD:
        status_text = f"Anomaly: {loss_display}"
        color = (0, 0, 255)
        anomaly_count += 1

        # --- Anomaly Localization (using ONNX output - still in NumPy for this part) ---
        original_processed_np = processed_frame_onnx[0, :, :, 0]
        reconstructed_frame_2d = reconstructed_frame_onnx[0, :, :, 0]
        squared_diff_map = (original_processed_np - reconstructed_frame_2d)**2

        error_map_display = cv2.normalize(squared_diff_map, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
        error_map_display_resized = cv2.resize(
            error_map_display, (original_frame_color.shape[1], original_frame_color.shape[0]),
            interpolation=cv2.INTER_LINEAR
        )
        cv2.imwrite(f"anomaly_maps/frame_{frame_count_processed:05d}_error_map_onnx.jpg", error_map_display_resized)

        _, binary_error_mask = cv2.threshold(
            (squared_diff_map * 255).astype(np.uint8),
            (ANOMALY_REGION_PIXEL_THRESHOLD * 255),
            255,
            cv2.THRESH_BINARY
        )
        binary_error_mask_resized = cv2.resize(
            binary_error_mask, (original_frame_color.shape[1], original_frame_color.shape[0]),
            interpolation=cv2.INTER_NEAREST
        )

        contours, _ = cv2.findContours(
            binary_error_mask_resized, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
        )

        for contour in contours:
            if cv2.contourArea(contour) > 50:
                x, y, w, h = cv2.boundingRect(contour)
                overlay = display_frame.copy()
                cv2.rectangle(overlay, (x, y), (x + w, y + h), (0, 0, 255), -1)
                alpha = 0.50
                display_frame = cv2.addWeighted(overlay, alpha, display_frame, 1 - alpha, 0)
        # --- End Anomaly Localization ---

    cv2.putText(display_frame, status_text, (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    if status_text.startswith("Anomaly"):
        cv2.imwrite(f"output_frames/frame_{frame_count_processed:05d}.jpg", display_frame)
    else:
        cv2.imwrite(f"output_frames/frame_{frame_count_processed:05d}.jpg", display_frame)

    cv2.imshow("Anomaly Detection (ONNX)", display_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    if frame_count_processed % 100 == 0:
        print(f"Processed {frame_count_processed} frames, found {anomaly_count} anomalies so far (ONNX).")

cap.release()
cv2.destroyAllWindows()

print(f"\nProcessing complete (ONNX)!")
print(f"Total frames processed: {frame_count_processed}")
print(f"Total anomalies detected: {anomaly_count}")
if frame_count_processed > 0:
    anomaly_rate = (anomaly_count / frame_count_processed * 100)
    print(f"Anomaly rate: {anomaly_rate:.2f}%")
else:
    print("No frames were processed, cannot calculate anomaly rate.")

ONNX model loaded successfully.
Expected input shape for 'encoder_conv1_input': ['unk__129', 232, 232, 1]
Using device: ['CUDAExecutionProvider', 'CPUExecutionProvider']
Total frames in video: 200
Frame 1 Loss (ONNX): 0.00275
Frame 2 Loss (ONNX): 0.00274
Frame 3 Loss (ONNX): 0.00278
Frame 4 Loss (ONNX): 0.00272
Frame 5 Loss (ONNX): 0.00272
Frame 6 Loss (ONNX): 0.00278
Frame 7 Loss (ONNX): 0.00280
Frame 8 Loss (ONNX): 0.00282
Frame 9 Loss (ONNX): 0.00287
Frame 10 Loss (ONNX): 0.00281
Frame 11 Loss (ONNX): 0.00278
Frame 12 Loss (ONNX): 0.00273
Frame 13 Loss (ONNX): 0.00287
Frame 14 Loss (ONNX): 0.00271
Frame 15 Loss (ONNX): 0.00267
Frame 16 Loss (ONNX): 0.00268
Frame 17 Loss (ONNX): 0.00273
Frame 18 Loss (ONNX): 0.00272
Frame 19 Loss (ONNX): 0.00271
Frame 20 Loss (ONNX): 0.00279
Frame 21 Loss (ONNX): 0.00272
Frame 22 Loss (ONNX): 0.00265
Frame 23 Loss (ONNX): 0.00270
Frame 24 Loss (ONNX): 0.00269
Frame 25 Loss (ONNX): 0.00274
Frame 26 Loss (ONNX): 0.00262
Frame 27 Loss (ONNX): 0.00257
Fr